### Session

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.language_models import FakeListChatModel
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import trim_messages, HumanMessage


class PrintOutputCallback(BaseCallbackHandler):
    def on_chat_model_start(self, serialized, messages, **kwargs):
        print(f"Amount of input messages: {len(messages)}")


sessions = {}
handler = PrintOutputCallback()
llm = FakeListChatModel(responses=["ai1", "ai2", "ai3"])

def get_session_history(session_id: str):
    if session_id not in sessions:
        sessions[session_id] = InMemoryChatMessageHistory()
    return sessions[session_id]

trimmer = trim_messages(
    max_tokens=1,
    strategy="last",
    token_counter=len,
    include_system=True,
    start_on="human",
)

raw_chain = trimmer | llm
chain = RunnableWithMessageHistory(raw_chain, get_session_history)

config = {"callbacks": [PrintOutputCallback()], "configurable": {"session_id": "1"}}
_ = chain.invoke(
    [HumanMessage("Hi!")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

_ = chain.invoke(
    [HumanMessage("How are you?")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

Amount of input messages: 1
History length: 2
Amount of input messages: 1
History length: 4


### Checkpoint

In [2]:
from langgraph.graph import MessageGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage
from langgraph.graph import START, END


def test_node(state):
   # ignore the last message since it's an input one
   print(f"History length = {len(state[:-1])}")
   return [AIMessage(content="Hello!")]

builder = MessageGraph()
builder.add_node("test_node", test_node)
builder.add_edge(START, "test_node")
builder.add_edge("test_node", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}})
_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-b"}})
_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}})

History length = 0
History length = 0
History length = 2


In [3]:
checkpoints = list(memory.list(config={"configurable": {"thread_id": "thread-a"}}))
for check_point in checkpoints:
  print(check_point.config["configurable"]["checkpoint_id"])

1f078cde-5c4a-6f14-8004-88c63f1b3cb5
1f078cde-5c46-64f1-8003-2f44bc593ba0
1f078cde-5c41-6b87-8002-ae88a1704093
1f078cde-5c20-6527-8001-850e059dac9f
1f078cde-5c17-6b54-8000-8b150f2e8e50
1f078cde-5c0f-6164-bfff-8a524383df91


In [4]:
checkpoint_id = checkpoints[-1].config["configurable"]["checkpoint_id"]
_ = graph.invoke(
    [HumanMessage(content="test")],
    config={"configurable": {"thread_id": "thread-a", "checkpoint_id": checkpoint_id}})

History length = 0


In [5]:
checkpoint_id = checkpoints[-3].config["configurable"]["checkpoint_id"]
_ = graph.invoke(
    [HumanMessage(content="test")],
    config={"configurable": {"thread_id": "thread-a", "checkpoint_id": checkpoint_id}})

History length = 2
